# KnapSackEvolutive

Experimento de resolução do problema da mochila usando algoritmos genéticos

## Dependências

In [599]:
import random
import time
import pandas as pd

## Variáveis iniciais

In [600]:
# Lista de itens: ID, Nome do item, Valor (R$), Peso (kg)
items = [
    (1, "Barraca", 150.0, 3.5),
    (2, "Saco de dormir", 100.0, 2.0),
    (3, "Isolante térmico", 50.0, 0.5),
    (4, "Colchão inflável", 80.0, 1.0),
    (5, "Lanterna", 30.0, 0.2),
    (6, "Kit de primeiros socorros", 20.0, 0.5),
    (7, "Repelente de insetos", 15.0, 0.1),
    (8, "Protetor solar", 20.0, 0.2),
    (9, "Canivete", 10.0, 0.1),
    (10, "Mapa e bússola", 25.0, 0.3),
    (11, "Garrafa de água", 15.0, 1.8),
    (12, "Filtro de água", 50.0, 0.5),
    (13, "Comida (ração liofilizada)", 50.0, 3.0),
    (14, "Fogão de camping", 70.0, 1.5),
    (15, "Botijão de gás", 30.0, 1.2),
    (16, "Prato, talheres e caneca", 20.0, 0.5),
    (17, "Roupas (conjunto)", 80.0, 1.5),
    (18, "Calçados (botas)", 120.0, 2.0),
    (19, "Toalha", 20.0, 0.5),
    (20, "Kit de higiene pessoal", 30.0, 0.5)
]

# Capacidade máxima da mochila em kg
max_weight = 15.0

## Parâmetros do algoritmo genético

### Tamanho do cromossomo

In [601]:
chromosome_length = len(items)

É o numero de cromossomos em cada indivíduo, é fixo no tamanho da lista disponível

### População inicial

In [602]:
population_size = 1000

Número de indivíduos na população inicial

**Considerações**

Diversidade Inicial: Um tamanho maior da população aumenta a probabilidade de <mark>exploração</mark> uma parte mais ampla do espaço de busca no início, mas populações maiores podem retardar a convergência.

Convergência: Populações menores tendem a convergir mais rapidamente, favorecendo a <mark>exploitação</mark> de regiões específicas, mas podem ficar presas em mínimos locais.

Custo Computacional: Populações maiores aumentam o número de avaliações de fitness por geração, o que pode aumentar o tempo de execução.

### Número de Gerações

In [603]:
num_generations = 50

É o número de iterações que o algoritmo genético realizará, a cada geração, a população de indivíduos evolui com base em operadores como crossover (recombinação de pais) e mutação.

**Considerações**

Mais gerações: Pode levar a um processo de evolução mais demorado, mas também oferece mais oportunidades para o algoritmo encontrar uma solução melhor.

Menos gerações: Pode acelerar a execução, mas também aumenta o risco de o algoritmo não ter tempo suficiente para explorar adequadamente o espaço de soluções.

### Tamanho do torneio

In [604]:
tournament_size = 15

Tamanho do torneio, ou número de individuos selecionados para reprodução

**Brevemente, como Funciona a Seleção por Torneio:**

Subconjunto Aleatório: Para cada seleção, o algoritmo escolhe aleatoriamente um grupo de indivíduos da população. O número de indivíduos nesse grupo é definido por *tournament_size*.

Competição: O indivíduo com o melhor valor de fitness dentro desse grupo é selecionado.

Repetição: O processo é repetido até que o número desejado de indivíduos seja selecionado.

**Considerações**

Valores pequenos (e.g., tournsize=2):

A seleção se torna mais aleatória. Há maior diversidade na próxima geração, pois indivíduos com fitness baixos ainda têm chances razoáveis de serem selecionados. Pode ser útil no início do algoritmo, onde a <mark>exploração</mark> é mais importante.

Valores grandes (e.g., tournsize=10):

A seleção se torna mais elitista, favorecendo indivíduos com fitness altos. Isso acelera a convergência, mas pode levar o algoritmo a mínimos locais se a diversidade for insuficiente. Pode seu útil após achar as áreas mais favoráveis do problema pois favorece a <mark>exploitação</mark> do local.

Valor igual ao tamanho da população:

A seleção se torna totalmente determinística, pois sempre escolherá o indivíduo com o melhor fitness na população.

### Taxa de Crossover

In [605]:
crossover_probability = 0.5

Define a probabilidade de que dois indivíduos (pais) se cruzem para gerar um ou mais filhos. No crossover, partes dos genes de dois indivíduos são trocadas para criar novos indivíduos.

**Considerações**

Taxa de Crossover alta (e.g., 0.9): Mais cruzamentos entre indivíduos, promovendo mais exploração e diversidade. Pode acelerar a busca por boas soluções, mas também pode resultar em soluções mais diversificadas e não tão refinadas.

Taxa de Crossover baixa (e.g., 0.1): Menos recombinação entre indivíduos, promovendo mais exploração local das soluções, o que pode ser bom para refinar uma solução existente, mas pode limitar a diversidade.

### Taxa de Mutação

In [606]:
mutation_probability = 0.2

Define a probabilidade de que um indivíduo sofra uma mutação em seus genes e introduz novas variações aleatórias no processo evolutivo.

**Considerações**

Taxa de Mutação alta (e.g., 0.5): Mais mutações, o que pode resultar em maior <mark>exploração</mark> do espaço de busca. No entanto, uma taxa muito alta pode fazer o algoritmo perder boas soluções já encontradas.

Taxa de Mutação baixa (e.g., 0.01): Menos mutações, favorecendo a <mark>exploitação</mark> local de soluções e uma busca mais direcionada, mas com risco de convergência prematura (ficar preso em mínimos locais).

### Probabilidade individual de mutação

In [607]:
individual_probability = 0.05

Probabilidade de mutação de cada gene em um indivíduo (bit flip)

**Considerações**

Valores baixos (e.g., 0.01): Introduzem mutação esporádica, preservando a maior parte dos genes originais. É útil para pequenas alterações no indivíduo.

Valores altos (e.g., 0.5): Introduzem mutação mais intensa, alterando mais genes. Pode ajudar a escapar de mínimos locais, mas corre o risco de desestabilizar soluções já boas.

### Melhores indivíduos

In [608]:
best_individuals = 5

Número de indivíduos selecionados ao final

## Funções

### Fitness

In [609]:
# Calcula o valor total dos itens na mochila e penaliza se o peso máximo for excedido
def fitness_function(individual):
    total_value = 0
    total_weight = 0
    for gene, item in zip(individual, items):
        if gene == 1:
            total_value += item[2]
            total_weight += item[3]
    return total_value if total_weight <= max_weight else -1

### Populacionais

In [610]:
# Geração inicial da população
def population_generate(population_size, chromosome_length):
    return [
        (random.choices([0, 1], k=chromosome_length), 0) for _ in range(population_size)
    ]


# Avaliação da população
def population_evaluate(population, fitness_func):
    for i, (chromosome, _) in enumerate(population):
        population[i] = (chromosome, fitness_func(chromosome))
    return population

### Seleção por divisão tripla (**B**, **T**, **R**)

In [611]:
# Seleção de indivíduos (elitismo, torneio, aleatório)
def population_select_triple(population, tournament_size):
    B = sorted(population, key=lambda x: x[1], reverse=True)[:tournament_size]  # Elitismo
    T = [random.choice(sorted(random.sample(population, tournament_size), key=lambda x: x[1], reverse=True)) for _ in range(tournament_size)] # Torneio
    R = random.sample(population, tournament_size) # Aleatório
    return B, T, R

## Operadores

### Reprodução

In [612]:
# Operador de reprodução
def operator_mating(parent1, parent2, crossover_probability):
    if random.random() < crossover_probability:
        child1 = [random.choice(genes) for genes in zip(parent1, parent2)]
        child2 = [random.choice(genes) for genes in zip(parent1, parent2)]
    else:
        child1, child2 = parent1[:], parent2[:]
    return child1, child2

### Mutação

In [613]:
# Operador de mutação
def operator_xmen(individual, mutation_probability, individual_probability):
    if random.random() < mutation_probability:
        return [1 - gene if random.random() < individual_probability else gene for gene in individual]
    return individual[:]

## Algoritmo & Execução

In [614]:
# Geração inicial e avaliação inicial da população
population = population_generate(population_size, chromosome_length)
population = population_evaluate(population, fitness_function)

# Registra o tempo de início
start_time = time.time()

# Loop principal do algoritmo genético com o número de gerações
for generation in range(num_generations):

    # Seleção dos três grupos
    B, T, R = population_select_triple(population, tournament_size)

    # Reprodução nos trê grupos
    offspring_B = []
    for _ in range(len(B) // 2):
        p1, p2 = random.sample(B, 2)
        child1, child2 = operator_mating(p1[0], p2[0], crossover_probability)
        offspring_B.append((child1, 0))
        offspring_B.append((child2, 0))

    offspring_T = []
    for _ in range(len(T) // 2):
        p1, p2 = random.sample(T, 2)
        child1, child2 = operator_mating(p1[0], p2[0], crossover_probability)
        offspring_T.append((child1, 0))
        offspring_T.append((child2, 0))

    offspring_R = []
    for _ in range(len(R) // 2):
        p1, p2 = random.sample(R, 2)
        child1, child2 = operator_mating(p1[0], p2[0], crossover_probability)
        offspring_R.append((child1, 0))
        offspring_R.append((child2, 0))

    # Mutação nos três grupos
    xmen_B = [(operator_xmen(ind[0], mutation_probability, individual_probability), ind[1]) for ind in B]

    xmen_T = [(operator_xmen(ind[0], mutation_probability, individual_probability), ind[1]) for ind in T]

    xmen_R = [(operator_xmen(ind[0], mutation_probability, individual_probability), ind[1]) for ind in R]

    # Combinação dos grupos
    combined = B + T + R

    # Reprodução cruzada
    offspring_BTR = []
    for _ in range(len(combined) // 2):
        p1, p2 = random.sample(combined, 2)
        child1, child2 = operator_mating(p1[0], p2[0], crossover_probability)
        offspring_BTR.append((child1, 0))
        offspring_BTR.append((child2, 0))

    # Mutação cruzada
    xmen_BTR = [(operator_xmen(ind[0], mutation_probability, individual_probability), 0) for ind in combined]

    descendants = offspring_B + offspring_T + offspring_R + offspring_BTR + xmen_B + xmen_T + xmen_R + xmen_BTR

    # Reavaliação da população
    population = population_evaluate(descendants, fitness_function)

    # Ordenação e manutenção dos melhores
    population = sorted(population, key=lambda x: x[1], reverse=True)[:population_size]

# Registra o tempo de término
end_time = time.time()

# Calcula o tempo total de execução
execution_time = end_time - start_time

# Seleção dos melhores indivíduos finais
top_individuals = sorted(population, key=lambda x: x[1], reverse=True)[:best_individuals]

## Resultados

In [615]:
# Lista melhores indivíduos
print("Melhores indivíduos:")
for ind in top_individuals:
    selected_items = [items[i] for i, gene in enumerate(ind[0]) if gene == 1]
    total_value = sum(item[2] for item in selected_items)
    total_weight = sum(item[3] for item in selected_items)
    print(f"Cromossomo: {ind[0]}, Fitness: {ind[1]}, Valor Total: {total_value:.2f}, Peso Total: {total_weight:.2f}")

Melhores indivíduos:
Cromossomo: [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1], Fitness: 870.0, Valor Total: 870.00, Peso Total: 14.90
Cromossomo: [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1], Fitness: 870.0, Valor Total: 870.00, Peso Total: 14.90
Cromossomo: [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1], Fitness: 870.0, Valor Total: 870.00, Peso Total: 14.90
Cromossomo: [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1], Fitness: 870.0, Valor Total: 870.00, Peso Total: 14.90
Cromossomo: [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1], Fitness: 870.0, Valor Total: 870.00, Peso Total: 14.90


## Resultados (Melhor indivíduo)

In [616]:
# Seleciona o melhor indivíduo.
best_individual = top_individuals[0]
print("Cromossomo:", best_individual)

Cromossomo: ([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1], 870.0)


In [617]:
# Identifica os itens selecionados no melhor indivíduo.
selected_items = [items[i] for i, gene in enumerate(best_individual[0]) if gene == 1]
total_value = sum(item[2] for item in selected_items)
total_weight = sum(item[3] for item in selected_items)

In [618]:
# Criação do DataFrame com os itens selecionados pelo melhor indivíduo
selected_items_data = [{
    'Nome do item': item[1],
    'Valor (R$)': item[2],
    'Peso (kg)': item[3]
} for item in selected_items]

df_selected_items = pd.DataFrame(selected_items_data)

print("Itens selecionados:")
df_selected_items.head(len(items))

Itens selecionados:


,Nome do item,Valor (R$),Peso (kg)
0,Barraca,150.0,3.5
1,Saco de dormir,100.0,2.0
2,Isolante térmico,50.0,0.5
3,Colchão inflável,80.0,1.0
4,Lanterna,30.0,0.2
5,Repelente de insetos,15.0,0.1
6,Protetor solar,20.0,0.2
7,Canivete,10.0,0.1
8,Mapa e bússola,25.0,0.3
9,Filtro de água,50.0,0.5


In [619]:
# Imprime o valor total dos itens na mochila.
print(f"Valor total: R${sum(item[2] for item in selected_items):.2f}")

Valor total: R$870.00


In [620]:
# Imprime o peso total dos itens na mochila.
print(f"Peso total: {sum(item[3] for item in selected_items):.2f}kg")

Peso total: 14.90kg


In [621]:
# Imprime o tempo de execução
print(f"Tempo de execução (evolução somente): {execution_time:.4f} segundos")

Tempo de execução (evolução somente): 0.0410 segundos
